# Component: Model I/O

In [9]:
!pip install langchain langchain-openai langchain-huggingface

In [10]:
from dotenv import load_dotenv
load_dotenv()

True

### PromptTemplate

In [11]:
# PromptTemplate
from langchain import PromptTemplate

template = '{product}을 홍보하기 위한 재미있고, 신선한 광고 문구를 작성해 주세요.'

prompt = PromptTemplate(
    template=template,
    input_variables=['product']
)

print(prompt.format(product='수박'))
print(prompt.format(product='단감'))

수박을 홍보하기 위한 재미있고, 신선한 광고 문구를 작성해 주세요.
단감을 홍보하기 위한 재미있고, 신선한 광고 문구를 작성해 주세요.


In [12]:
# FewShotPromptTemplate: n-shot 프롬프팅 적용
from langchain.prompts import FewShotPromptTemplate

examples = [
    {"question": "2 + 2는 무엇인가요?", "answer": "2 + 2 = 4"},
    {"question": "3 + 4는 무엇인가요?", "answer": "3 + 4 = 7"},
]

example_prompt = PromptTemplate(
    template="Q: {question}\nA: {answer}",
    input_variables=["question", "answer"]
)

fewshot_prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    prefix="다음 계산 문제를 해결하세요.",
    suffix="Q: {question}은 무엇인가요?\nA:",
    input_variables=['question']
)

print(fewshot_prompt.format(question='325 + 111'))

다음 계산 문제를 해결하세요.

Q: 2 + 2는 무엇인가요?
A: 2 + 2 = 4

Q: 3 + 4는 무엇인가요?
A: 3 + 4 = 7

Q: 325 + 111은 무엇인가요?
A:


In [13]:
# ChatPromptTemplate: System, Human(user), AI(assistant) 유형별 메시지 작성
from langchain.prompts.chat import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate, AIMessagePromptTemplate

sys_msg = SystemMessagePromptTemplate.from_template('너는 아주 친절한 챗봇이야')
hm_msg = HumanMessagePromptTemplate.from_template('질문: {question}')
msg = ChatPromptTemplate.from_messages([sys_msg, hm_msg])
print(msg)

msg.format_messages(question='AI를 배우려면 뭐부터 시작해야 할까?')

input_variables=['question'] input_types={} partial_variables={} messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='너는 아주 친절한 챗봇이야'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['question'], input_types={}, partial_variables={}, template='질문: {question}'), additional_kwargs={})]


[SystemMessage(content='너는 아주 친절한 챗봇이야', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='질문: AI를 배우려면 뭐부터 시작해야 할까?', additional_kwargs={}, response_metadata={})]

### OutputParser

In [ ]:
# CommaSeparatedListOutputParser
from langchain import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain.output_parsers import CommaSeparatedListOutputParser

output_parser = CommaSeparatedListOutputParser()
format_instructions = output_parser.get_format_instructions()

prompt_tpl = PromptTemplate(
    template="{subject} 5개의 팀을 알려주세요.\n형식 지정: {format}",
    input_variables=['subject'],                        # 사용자 입력 변수
    partial_variables={'format': format_instructions}   # 고정 설정 변수
)

In [15]:
query = '한국의 걸그룹'
prompt = prompt_tpl.format(subject=query)
prompt

'한국의 걸그룹 5개의 팀을 알려주세요.\n형식 지정: Your response should be a list of comma separated values, eg: `foo, bar, baz` or `foo,bar,baz`'

In [16]:
model = ChatOpenAI(
    model_name='gpt-4o-mini',
    temperature=0.1,
    max_tokens=2048
)

response = model.invoke(prompt)

In [17]:
print(response.content)
print(type(response.content))

BLACKPINK, TWICE, Red Velvet, ITZY, (G)I-DLE
<class 'str'>


In [18]:
print(output_parser.parse(response.content))
print(type(output_parser.parse(response.content)))

['BLACKPINK', 'TWICE', 'Red Velvet', 'ITZY', '(G)I-DLE']
<class 'list'>


In [ ]:
# JSONOutputParser
from langchain_core.output_parsers import JsonOutputParser

# PromptTemplate > LLM > JSONOutputParser
# ㄴ {subject} 관련 책 {n}권을 보여주세요. (+ json 형식 지정)
json_parser = JsonOutputParser()

prompt_tpl = PromptTemplate(
    template="{subject} 관련 책 {n}권을 보여주세요. 실제 존재하는 책 정보만 작성하고, 절대 지어내지 마세요.\n{format_instructions}",
    input_variables=['subject', 'n'],
    partial_variables={'format_instructions': json_parser.get_format_instructions()}
)

llm = ChatOpenAI(model_name='gpt-4o-mini')

prompt = prompt_tpl.format(subject='AI', n=3)
ai_message = llm.invoke(prompt)

In [20]:
output = json_parser.parse(ai_message.content)
print(output)
print(type(output[0]))

[{'title': 'Artificial Intelligence: A Guide to Intelligent Systems', 'author': 'Michael Negnevitsky', 'published_year': 2011, 'publisher': 'Addison-Wesley'}, {'title': 'Deep Learning', 'author': 'Ian Goodfellow, Yoshua Bengio, Aaron Courville', 'published_year': 2016, 'publisher': 'MIT Press'}, {'title': 'Life 3.0: Being Human in the Age of Artificial Intelligence', 'author': 'Max Tegmark', 'published_year': 2017, 'publisher': 'Knopf'}]
<class 'dict'>


In [ ]:
# chain 들어가기 (맛보기) --- lcel
chain = prompt_tpl | llm | json_parser
output = chain.invoke(input={'subject': 'NLP', 'n': 3})
print(output)

{'books': [{'title': 'Speech and Language Processing', 'authors': ['Daniel Jurafsky', 'James H. Martin'], 'publication_year': 2020, 'ISBN': '978-0134845623'}, {'title': 'Natural Language Processing with Python', 'authors': ['Steven Bird', 'Ewan Klein', 'Edward Loper'], 'publication_year': 2009, 'ISBN': '978-0596105646'}, {'title': 'Deep Learning for Natural Language Processing', 'authors': ['Palash Goyal', 'Lakhendra Kumar', 'Bharat Rawal', 'Dinesh Kumar'], 'publication_year': 2021, 'ISBN': '978-3030645491'}]}


##### HuggingFace model 활용

In [21]:
import os
HF_TOKEN = os.getenv('HF_TOKEN')

In [22]:
from langchain_huggingface import HuggingFaceEndpoint, ChatHuggingFace

endpoint = HuggingFaceEndpoint(
    repo_id='MLP-KTLim/llama-3-Korean-Bllossom-8B',
    task='text-generation',
    max_new_tokens=1024,
    huggingfacehub_api_token=HF_TOKEN
)

hf_model = ChatHuggingFace(
    llm=endpoint,
    verbose=True
)

hf_model.invoke('아침으로 사과를 먹으면 좋을까?')

AIMessage(content='아침에 사과를 먹는 것은 건강에 좋은 선택일 수 있지만, 개인의 건강 상태와 식습관에 따라 다를 수 있습니다. 다음은 아침에 사과를 먹는 장점과 단점입니다.\n\n### 장점\n\n1. **영양 공급**: 사과는 비타민 A, 비타민 C, 칼륨, 포타슘 등 다양한 영양소를 함유하고 있어 몸의 기능을 지원하는 데 도움이 됩니다.\n2. **섬유질**: 사과는 높은 섬유질 함량을 가지고 있어 소화가 잘 되고, 장 건강을 유지하는 데 도움이 됩니다.\n3. **포도당 공급**: 사과는 자연스럽게 포도당을 공급하여 에너지를 제공합니다. 이는 아침 식사를 통해 하루 첫 번째 에너지원으로 작용할 수 있습니다.\n4. **채식 섭취**: 사과는 과일로, 아침에 채식 섭취를 늘리는 데 도움이 됩니다. 이는 비타민, 미네랄, 항산화 물질 등 다양한 건강 혜택을 제공합니다.\n\n### 단점\n\n1. **섬유질 문제**: 사과는 섬유질이 높아, 소화가 잘 되지 않을 경우 변비나 소화 문제를 일으킬 수 있습니다.\n2. **포도당 급증**: 사과는 포도당이 많이 함유되어 있어, 혈당 수치가 급격히 상승할 수 있습니다. 이는 당뇨병 환자나 고혈당을 우려하는 사람들에게 주의가 필요합니다.\n3. **알레르기**: 일부 사람들은 사과에 알레르기가 있을 수 있습니다. 알레르기 반응은 호흡기, 피부, 식도 등 다양한 부분에서 나타날 수 있습니다.\n4. **영양 불균형**: 사과만을 아침 식사로 삼는 것은 다른 영양소가 부족할 수 있습니다. 예를 들어, 단백질, 지방, 무기물 등의 균형이 맞지 않을 수 있습니다.\n\n### 결론\n\n아침에 사과를 먹는 것은 건강에 좋은 선택일 수 있지만, 개인의 건강 상태와 식습관에 따라 다를 수 있습니다. 사과를 먹기 전에 다음과 같은 점을 고려하는 것이 좋습니다:\n\n- 사과를 먹기 전에 충분한 물을 마시세요.\n- 사과를 섭취할 때 다른 음식과 함께 먹는 것이 좋습니다. 예를 들어, 사과와 함께 토스트나 요거트

##### Model Laboratory

In [ ]:
from langchain.model_laboratory import ModelLaboratory

model_lab = ModelLaboratory.from_llms([model, hf_model])    # 두 모델의 응답 비교
model_lab.compare('아침에 사과를 먹는 것의 효과를 알려줘')

Input:
아침에 사과를 먹는 것의 효과를 알려줘

client=<openai.resources.chat.completions.completions.Completions object at 0x00000152B8CC38C0> async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x00000152B91E5550> root_client=<openai.OpenAI object at 0x00000152B88507A0> root_async_client=<openai.AsyncOpenAI object at 0x00000152B8D8F170> model_name='gpt-4o-mini' temperature=0.1 model_kwargs={} openai_api_key=SecretStr('**********') max_tokens=2048
아침에 사과를 먹는 것은 여러 가지 건강상의 이점을 제공합니다. 다음은 그 효과들입니다:

1. **영양소 공급**: 사과는 비타민 C, 식이섬유, 항산화 물질이 풍부하여 면역 체계를 강화하고 건강한 피부를 유지하는 데 도움을 줍니다.

2. **소화 개선**: 사과에 포함된 식이섬유는 장 건강을 촉진하고 소화를 개선하는 데 도움을 줄 수 있습니다. 특히, 사과의 껍질에 있는 펙틴은 장내 유익한 세균의 성장을 도와줍니다.

3. **체중 관리**: 사과는 칼로리가 낮고 수분 함량이 높아 포만감을 주어 체중 관리에 도움이 될 수 있습니다. 아침에 사과를 먹으면 과식을 방지하는 데 도움이 될 수 있습니다.

4. **에너지 공급**: 사과는 자연적인 당분을 포함하고 있어 아침에 에너지를 공급해 주며, 하루를 시작하는 데 필요한 활력을 제공합니다.

5. **심혈관 건강**: 사과에 포함된 항산화 물질과 식이섬유는 심혈관 건강을 개선하고 나쁜 콜레스테롤 수치를 낮추는 데 도움을 줄 수 있습니다.

6. **혈당 조절**: 사과는 혈당 수치를